# Jane Street - Submission
This is the actual notebook submitted to the competition. First, we load the trained models and use them to implement the `predict()` function.

In [1]:
import json
import os

import pandas as pd

from tensorflow import keras

# load the model
model_folder = os.path.join(os.pardir, "input", "jane-street-tpu")

with open(os.path.join(model_folder, "model.json"), "r") as f:
    model_json = f.read()

model = keras.models.model_from_json(model_json)
model.load_weights(os.path.join(model_folder, "model.h5"))

with open(os.path.join(model_folder, "params.json")) as file:
    params = json.loads(file.read())

WINDOW_SIZE = params["window_size"]


# get the optimal threshold from the utility notebook
util_file = os.path.join(os.pardir, "input", "jane-street-utility", "threshold.json")

with open(util_file) as file:
    threshold_json = json.loads(file.read())

THRESHOLD = float(threshold_json["threshold"])


# load stats dictionary to get the nan value
stats_file = os.path.join(os.pardir, "input", "jane-street-data", "stats.json")

with open(stats_file) as file:
    stats = json.loads(file.read())

NAN_VALUE = float(stats["nan_value"])


# function to return predictions for the current trade
def predict(df):
    if len(df) < WINDOW_SIZE:
        return 0
    series = df.iloc[-WINDOW_SIZE:].to_numpy().reshape(1, WINDOW_SIZE, 130)
    pred = model(series, training=False).numpy().item()
    
    return 1 if pred > THRESHOLD else 0

Then we use the Jane Street API to make the predictions. In this setup the iterator `iter_test` yields single samples `test_df` in chronological order. The predict function needs to decide whether to accept or reject the trade at this time-point using only the current sample and the previous samples stored in `hist_df`. Set `pred_df` to 1 to accept the trade or to 0 to reject the trade.

In [2]:
import janestreet

env = janestreet.make_env()
iter_test = env.iter_test()

# initialize things with first sample
sample, pred = next(iter_test)

feat_cols = [c for c in sample.columns if "feature" in c]
last_date = sample["date"].item() 

df = sample[feat_cols].fillna(NAN_VALUE)

pred.action = 0
env.predict(pred)


# loop over remaining samples
for (sample, pred) in iter_test:
    # if new date, clear df
    new_date = sample["date"].item() 
    if new_date != last_date:
        last_date = new_date
        df.drop(df.index, inplace=True)
        
    # append new sample to df
    sample.fillna(NAN_VALUE, inplace=True)
    df = df.append(sample[feat_cols], ignore_index=True)
    
    # accept/reject current trade
    pred.action = predict(df) if sample["weight"].item() > 0.0 else 0
    env.predict(pred)